In [1]:
import pandas as pd
import lxml.etree 
import csv
import tarfile
import os
from tqdm import tqdm

import glob

In [2]:


def getHeaderFromXSLT(xslt):
    root=xslt.getroot()
    
    headers=[]

    for item in root.findall('.//xsl:value-of',namespaces=root.nsmap):
        #ignore variables
        if (item.find('..').tag!='{http://www.w3.org/1999/XSL/Transform}variable'):
            el = item.attrib['select'].replace('.//','').replace('@','').replace('normalize-space(translate(','').replace(",'[¬]',''))",'').replace('$','').replace('/','__')
            if(el=='.'):
                #select the parent of the tag in order ot get select attribute
                el = item.find('..').attrib['select'].replace('.//','').replace('@','').replace('normalize-space(translate(','').replace(",'[¬]',''))",'').replace('$','').replace('/','__')
            headers.append(el)
    return headers


    



def readTar(file:str):
    with tarfile.open(file, "r:gz") as tar:
        # Go over each member
        for member in tar:
            if '.xml' in member.name:
                yield tar.extractfile(member).read()  

def writeRows(writer,tree,header,xslt):
    transform = lxml.etree.XSLT(xslt)
    transformed=transform(tree)
    lines=str(transformed).split('$end_line$')
    for line in lines:
        #write only if there are values in the line
        if (len(header)<=len(line)):
            writer.writerow(line.split('¬'))


def main():

    file='ORCID_2021_10_summaries.tar.gz'


    outdir='data'
    os.makedirs(outdir, exist_ok=True)

    xslt_files=[]
    headers=[]
    files=[]
    filenames=[]
    writers=[]

    for xslt_file in glob.glob('xslt/*.xsl'):
        xslt=lxml.etree.parse(xslt_file)
        xslt_files.append(xslt)
        name=os.path.basename(xslt_file).replace('.xsl','')
        filenames.append(name)
        f=open(os.path.join(outdir, "orcid_"+name+'.csv'), "w", encoding="utf8")
        files.append(f)
        headers.append(getHeaderFromXSLT(xslt))
        writers.append(csv.writer(f, lineterminator="\n"))


    #write header row
    for i in range(0,len(files)):
        writers[i].writerow(headers[i])


    import itertools

    for xml in tqdm(itertools.islice(readTar(file), 10000),total=10000):
    #for xml in tqdm(readTar(file),total=12000000):
        # Extract member
        tree = lxml.etree.fromstring(xml)
        for i in range(0,len(files)):
            writeRows(writers[i],tree,headers[i],xslt_files[i])
            

    for file in files:
        file.close()
            


100%|██████████| 10000/10000 [01:03<00:00, 157.48it/s]
